# Arduino Node with sensors and actuators

Remind that we want to develop a HW structure with RaspberryPi3 and Arduino 

<img src="./Images/1_nodes_schematics.png">

Bibliography:

course ROS by Anis Koubaa: https://github.com/aniskoubaa/ros_essentials_cpp

Course ROS by M. Lukman:https://www.udemy.com/course/mastering-mobile-robot-with-ros-ardunio-car-sensors-to-ros/learn/lecture/14385996#content

### Range sensor

From the different Ultrasonic range sensors: https://www.intorobotics.com/interfacing-programming-ultrasonic-sensors-tutorials-resources/

Develop our first program for Ultrasonic range sensor HC-SC04 (or SR05):

<img src="./Images/HC_SRF05.png">

Install the library Ultrasound by Erick Simoes from the library manager.

There are some exemples wit one and several ultrasonic sensors.

we use sensor_msgs/Range data type: http://docs.ros.org/melodic/api/sensor_msgs/html/msg/Range.html

We convert the code in the frame of ROS-Publisher code: (ROS_1HC_SR04_Pub.ino)

In [ ]:
/*
 * ROS Ultrasonic 1 unit HC-SR04
*/

#include <Ultrasonic.h>
#include <ros.h>
#include <sensor_msgs/Range.h>

Ultrasonic ultrasonic(12, 13);

ros::NodeHandle nh;

sensor_msgs::Range range_msg;

ros::Publisher pub_range_ultrasound("/ultrasound", &range_msg);

void setup() {

   nh.initNode();
   nh.advertise(pub_range_ultrasound);
}

void loop() {

    range_msg.range = ultrasonic.read();
    pub_range_ultrasound.publish(&range_msg);

    nh.spinOnce();
}

### IMU 9250

We can follow the tutorial: https://atadiat.com/en/e-ros-imu-and-arduino-how-to-send-to-ros/


Pub_imu.ino

In [ ]:
//rosrun rosserial_python serial_node.py tcp


#include <Wire.h>
#include <ros.h>
#include <sensor_msgs/Imu.h>
ros::NodeHandle nh;

sensor_msgs::Imu msg;
ros::Publisher pub("IMU_data", &msg);

const char* ssid     = "paradise1";
const char* password = "112233445";
IPAddress server(192,168,10,2);
const uint16_t serverPort = 11411;

// MPU6050 Slave Device Address
const uint8_t MPU6050SlaveAddress = 0x68;

// Select SDA and SCL pins for I2C communication 
const uint8_t scl = D1;
const uint8_t sda = D2;

// sensitivity scale factor respective to full scale setting provided in datasheet 
const uint16_t AccelScaleFactor = 16384;
const uint16_t GyroScaleFactor = 131;

// MPU6050 few configuration register addresses
const uint8_t MPU6050_REGISTER_SMPLRT_DIV   =  0x19;
const uint8_t MPU6050_REGISTER_USER_CTRL    =  0x6A;
const uint8_t MPU6050_REGISTER_PWR_MGMT_1   =  0x6B;
const uint8_t MPU6050_REGISTER_PWR_MGMT_2   =  0x6C;
const uint8_t MPU6050_REGISTER_CONFIG       =  0x1A;
const uint8_t MPU6050_REGISTER_GYRO_CONFIG  =  0x1B;
const uint8_t MPU6050_REGISTER_ACCEL_CONFIG =  0x1C;
const uint8_t MPU6050_REGISTER_FIFO_EN      =  0x23;
const uint8_t MPU6050_REGISTER_INT_ENABLE   =  0x38;
const uint8_t MPU6050_REGISTER_ACCEL_XOUT_H =  0x3B;
const uint8_t MPU6050_REGISTER_SIGNAL_PATH_RESET  = 0x68;

int16_t AccelX, AccelY, AccelZ, Temperature, GyroX, GyroY, GyroZ;

void setup() {
  nh.initNode();
  nh.advertise(pub);
  Serial.begin(9600);
  WiFi.begin(ssid, password);
  while (WiFi.status() != WL_CONNECTED) {
    Serial.print('.');
    delay(500);
  }
   Serial.print("IP address: ");
   Serial.println(WiFi.localIP());
   nh.getHardware()->setConnection(server, serverPort);

  Wire.begin(sda, scl);
  MPU6050_Init();
}

void loop() {
  double Ax, Ay, Az, T, Gx, Gy, Gz;
  
  Read_RawValue(MPU6050SlaveAddress, MPU6050_REGISTER_ACCEL_XOUT_H);
  
  //divide each with their sensitivity scale factor
  Ax = (double)AccelX/AccelScaleFactor;
  Ay = (double)AccelY/AccelScaleFactor;
  Az = (double)AccelZ/AccelScaleFactor;
  T = (double)Temperature/340+36.53; //temperature formula
  Gx = (double)GyroX/GyroScaleFactor;
  Gy = (double)GyroY/GyroScaleFactor;
  Gz = (double)GyroZ/GyroScaleFactor;

  Serial.print("Ax: "); Serial.print(Ax);
  Serial.print(" Ay: "); Serial.print(Ay);
  Serial.print(" Az: "); Serial.print(Az);
  Serial.print(" T: "); Serial.print(T);
  Serial.print(" Gx: "); Serial.print(Gx);
  Serial.print(" Gy: "); Serial.print(Gy);
  Serial.print(" Gz: "); Serial.println(Gz);
 //A is for accelerometer , G is for gyroscope
  if (nh.connected()) {
  Serial.println("--------------");
  msg.angular_velocity.x=Gx;
  msg.angular_velocity.y=Gy;
  msg.angular_velocity.z=Gz;

  msg.linear_acceleration.x=Ax;
  msg.linear_acceleration.y=Ay;
  msg.linear_acceleration.z=Az;
  msg.header.frame_id="Luqman";
  pub.publish(&msg);}
  delay(100);
  nh.spinOnce();
  delay(10);
}

void I2C_Write(uint8_t deviceAddress, uint8_t regAddress, uint8_t data){
  Wire.beginTransmission(deviceAddress);
  Wire.write(regAddress);
  Wire.write(data);
  Wire.endTransmission();
}

// read all 14 register
void Read_RawValue(uint8_t deviceAddress, uint8_t regAddress){
  Wire.beginTransmission(deviceAddress);
  Wire.write(regAddress);
  Wire.endTransmission();
  Wire.requestFrom(deviceAddress, (uint8_t)14);
  AccelX = (((int16_t)Wire.read()<<8) | Wire.read());
  AccelY = (((int16_t)Wire.read()<<8) | Wire.read());
  AccelZ = (((int16_t)Wire.read()<<8) | Wire.read());
  Temperature = (((int16_t)Wire.read()<<8) | Wire.read());
  GyroX = (((int16_t)Wire.read()<<8) | Wire.read());
  GyroY = (((int16_t)Wire.read()<<8) | Wire.read());
  GyroZ = (((int16_t)Wire.read()<<8) | Wire.read());
}

//configure MPU6050
void MPU6050_Init(){
  delay(150);
  I2C_Write(MPU6050SlaveAddress, MPU6050_REGISTER_SMPLRT_DIV, 0x07);
  I2C_Write(MPU6050SlaveAddress, MPU6050_REGISTER_PWR_MGMT_1, 0x01);
  I2C_Write(MPU6050SlaveAddress, MPU6050_REGISTER_PWR_MGMT_2, 0x00);
  I2C_Write(MPU6050SlaveAddress, MPU6050_REGISTER_CONFIG, 0x00);
  I2C_Write(MPU6050SlaveAddress, MPU6050_REGISTER_GYRO_CONFIG, 0x00);//set +/-250 degree/second full scale
  I2C_Write(MPU6050SlaveAddress, MPU6050_REGISTER_ACCEL_CONFIG, 0x00);// set +/- 2g full scale
  I2C_Write(MPU6050SlaveAddress, MPU6050_REGISTER_FIFO_EN, 0x00);
  I2C_Write(MPU6050SlaveAddress, MPU6050_REGISTER_INT_ENABLE, 0x01);
  I2C_Write(MPU6050SlaveAddress, MPU6050_REGISTER_SIGNAL_PATH_RESET, 0x00);
  I2C_Write(MPU6050SlaveAddress, MPU6050_REGISTER_USER_CTRL, 0x00);
}


## Odometry

<img src="./Images/3_odom1.png">

## Actuators

### DC motors

You can use the driver L298N

you can follow the tutorial: https://www.youtube.com/watch?v=c0L4gNKwjRw


<img src="./Images/L298N.png">

<img src="./Images/L298N_connections.png">

We can also use Arduino Motor Shield

you can follow the tutorial: https://www.instructables.com/id/Arduino-Motor-Shield-Tutorial/

The code is in DC_motor_rUBot.ino

<img src="./Images/Motor_shield.png">

We will use geometry_msgs/Twist: http://docs.ros.org/melodic/api/geometry_msgs/html/msg/Twist.html

Interesting codes in:

-with arduino mega: https://create.arduino.cc/projecthub/robinb/nox-a-house-wandering-robot-ros-652315
- with ESP8266: https://github.com/Reinbert/ros_diffdrive_robot
- ROS speciffic package: http://wiki.ros.org/simple_drive

We will develop our own controller using a L298N driver:

https://maker.pro/arduino/tutorial/how-to-use-arduino-with-robot-operating-system-ros

<img src="./Images/DC_motor_arduino.png">

The code in ROS is in ROS_DC_motor_rUBot.ino

The code integrated with the sensors is in:
- ROS_sensors2motor1.ino Including cmd_vel with Empty type message
- ROS_sensors2motor.ino Including cmd_vel with Twist type message
- ROS_sensors2motor2.ino Including cmd_vel with Twist type message and IKINE and functions

In [ ]:
/*
 * rosserial sensors range, imu, ultrasound to motor
 */

#include <ArduinoHardware.h>
#include <ros.h>
#include <std_msgs/String.h>
#include <std_msgs/Int32.h>
#include <geometry_msgs/Twist.h>


#define EN_L 9
#define IN1_L 10
#define IN2_L 11

#define EN_R 8
#define IN1_R 12
#define IN2_R 13


double w_r=0, w_l=0;

//wheel_rad is the wheel radius ,wheel_sep is
double wheel_rad = 0.0325, wheel_sep = 0.295;


ros::NodeHandle  nh;
int lowSpeed = 200;
int highSpeed = 50;
double speed_ang=0, speed_lin=0;


void velCb(const geometry_msgs::Twist& vel_msg){
  speed_ang = vel_msg.angular.z;
  speed_lin = vel_msg.linear.x;
  w_r = (speed_lin/wheel_rad) + ((speed_ang*wheel_sep)/(2.0*wheel_rad));
  w_l = (speed_lin/wheel_rad) - ((speed_ang*wheel_sep)/(2.0*wheel_rad));
}

ros::Subscriber<geometry_msgs::Twist> vel_sub("cmd_vel", velCb );
void Motors_init();
void MotorL(int Pulse_Width1);
void MotorR(int Pulse_Width2);

std_msgs::Int32 range_msg;
ros::Publisher range_pub("range", &range_msg);

int obstacle=6;

std_msgs::String imu_msg;
ros::Publisher imu_pub("imu", &imu_msg);

char rpy[]="R60P30Y30";

std_msgs::Int32 odom_msg;
ros::Publisher odom_pub("odom", &odom_msg);

int distance=10;

void setup()
{
  Motors_init();
  nh.initNode();
  nh.advertise(range_pub);
  nh.advertise(imu_pub);
  nh.advertise(odom_pub);
  nh.subscribe(vel_sub);
}

void loop()
{
  range_msg.data = obstacle;
  range_pub.publish( &range_msg );
  imu_msg.data = rpy;
  imu_pub.publish( &imu_msg );
  odom_msg.data = distance;
  odom_pub.publish( &odom_msg );
  
  MotorL(w_l*10);
  MotorR(w_r*10);
  nh.spinOnce();
  //delay(1); //low value (or eliminated) to avoid unsync!
}


void Motors_init(){

 pinMode(EN_L, OUTPUT);

 pinMode(EN_R, OUTPUT);

 pinMode(IN1_L, OUTPUT);

 pinMode(IN2_L, OUTPUT);

 pinMode(IN1_R, OUTPUT);

 pinMode(IN2_R, OUTPUT);

 digitalWrite(EN_L, LOW);

 digitalWrite(EN_R, LOW);

 digitalWrite(IN1_L, LOW);

 digitalWrite(IN2_L, LOW);

 digitalWrite(IN1_R, LOW);

 digitalWrite(IN2_R, LOW);

}

void MotorL(int Pulse_Width1){
 if (Pulse_Width1 > 0){

     analogWrite(EN_L, Pulse_Width1);

     digitalWrite(IN1_L, HIGH);

     digitalWrite(IN2_L, LOW);

 }

 if (Pulse_Width1 < 0){

     Pulse_Width1=abs(Pulse_Width1);

     analogWrite(EN_L, Pulse_Width1);

     digitalWrite(IN1_L, LOW);

     digitalWrite(IN2_L, HIGH);

 }

 if (Pulse_Width1 == 0){

     analogWrite(EN_L, Pulse_Width1);

     digitalWrite(IN1_L, LOW);

     digitalWrite(IN2_L, LOW);

 }

}


void MotorR(int Pulse_Width2){


 if (Pulse_Width2 > 0){

     analogWrite(EN_R, Pulse_Width2);

     digitalWrite(IN1_R, LOW);

     digitalWrite(IN2_R, HIGH);

 }

 if (Pulse_Width2 < 0){

     Pulse_Width2=abs(Pulse_Width2);

     analogWrite(EN_R, Pulse_Width2);

     digitalWrite(IN1_R, HIGH);

     digitalWrite(IN2_R, LOW);

 }

 if (Pulse_Width2 == 0){

     analogWrite(EN_R, Pulse_Width2);

     digitalWrite(IN1_R, LOW);

     digitalWrite(IN2_R, LOW);

 }

}

Note!: if we are using Arduino UNO board, the scketch uses 75% of the total flash memory available (32KB)
If we use a STM32 Nucleo 103RB, the scketch uses the 11% of the total flash memory available (128KB)

We will have to use STM32 Nucleo boards for the project!

In [ ]:
rosrun rosserial_python serial_node.py /dev/ttyACM0

In PC rUBot_navigation.py

In [ ]:
#!/usr/bin/env python

import rospy

from std_msgs.msg import String, Int32, Empty

# Initialise variables
range_msg=0
imu_msg=""
odom_msg=0

pub = None

#define function/functions to provide the required functionality
def rangeCb(r_msg): # msg could be any name
    global range_msg
    range_msg=r_msg.data
    rospy.loginfo(rospy.get_caller_id() + ": I heard %s", range_msg)  
    if range_msg<7:
        vel_msg=Empty() # message to be published
        rospy.loginfo("Too close!. I send Empty message to toggle LED")
        pub.publish(vel_msg)
    else:
        rospy.loginfo("Far enough!")

def imuCb(i_msg):
    global imu_msg
    imu_msg=i_msg.data
    rospy.loginfo(rospy.get_caller_id() + ": I heard %s", imu_msg)

def odomCb(o_msg):
    global odom_msg
    odom_msg=o_msg.data
    rospy.loginfo(rospy.get_caller_id() + ": I heard %s", odom_msg)
    
if __name__=='__main__':

    try:
        #Add here the name of the ROS node. Node names are unique named. Here without "anonimous"!
        rospy.init_node('rUBot_ctrl_node')
        #subscribe to a topic using rospy.Subscriber class
        sub_range=rospy.Subscriber('/range', Int32, rangeCb)
        sub_imu=rospy.Subscriber('/imu', String, imuCb)
        sub_odom=rospy.Subscriber('/odom', Int32, odomCb)
        #publish messages to a topic using rospy.Publisher class
        pub=rospy.Publisher('/cmd_vel', Empty, queue_size=10)
  
        rospy.spin()
      
    except rospy.ROSInterruptException:
        rospy.loginfo("node terminated.")

In [ ]:
rosrun hw_motion_plan rUBot_navigation.py

<img src="./Images/3_sensor2motor_terminal.png">

<img src="./Images/3_sensor2motor_rqt.png">

we can use a launch file to start all the nodes. Even we can start the "teleop_twist_keyboard" node to control the robot with the keyboard. 
We create a launch "rUBot_control.launch" file in the "hw_motion_plan" package:

In [ ]:
<launch>
    <node name="serial_node"        pkg="rosserial_python"      type="serial_node.py">
    <param name="port"              type="string"               value="/dev/ttyACM0"/>
    <param name="baud"              type="int"                  value="57600"/>
    </node>

    <node name="rUBot_ctrl_node"        pkg="hw_motion_plan"      type="rUBot_navigation.py">
    </node>

    <node name="teleop_twist_keyboard"        pkg="teleop_twist_keyboard"      type="teleop_twist_keyboard.py">
    </node>

</launch>

To start you can launch this file and the roscore will automatically start:

In [ ]:
roslaunch hw_motion_plan rUBot_control.launch